# Introduction 

This note evaluate smolLM-135M instruct model with llama.cpp

# Env and tools prepare

In [1]:
!cp -r /kaggle/input/llama-cpp-binary-compiled-with-gpu/llama.cpp /kaggle/working/

In [2]:
!chmod +x -R llama.cpp
%cd llama.cpp
!cp -r /usr/local/cuda-12.3/targets /usr/local/nvidia/ 

/kaggle/working/llama.cpp


# Run perplexity

Perplexity is the main evaluation metric for large language models (LLMs). It measures how well the model predicts a given sequence of tokens.
Formally, the perplexity of LLMs is the exponentiated average negative log-likelihood,  which means **perplexity smaller is better** 

Perplexity benchmark shows how accurate the model is 

In [ ]:

!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M-v2.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4_V2.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > ppl_Q4.log 2>&1
!/kaggle/working/llama.cpp/llama-perplexity -m  /kaggle/input/smollm-135m-instruct-gguf/gguf/fp16/1/SmolLM-135M-Instruc-fp16.gguf -f /kaggle/working/llama.cpp/wikitext-2-raw/wiki.test.raw --chunks 256 > pip_fp16.log 2>&1


In [ ]:
!tail -6 ppl_Q4_V2.log
!tail -6 ppl_Q4.log
!tail -6 pip_fp16.log

# Benchmarking the Inference Throughput and Memory Consumption 

Llama-bench shows performance and resource consumption. 
There are three types of test
- Prompt processing (pp): processing a prompt in batches (-p)
- Text generation (tg): generating a sequence of tokens (-n)
- Prompt processing + text generation (pg): processing a prompt followed by generating a sequence of tokens (-pg)

In the following test, we run text generation and prompt processing


In [ ]:
!/kaggle/working/llama.cpp/llama-bench -m /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M-v2.gguf -m /kaggle/input/smollm-135m-instruct-gguf/gguf/q4_k_m/1/SmolLM-135M-Instruct-Q4_K_M.gguf -m /kaggle/input/smollm-135m-instruct-gguf/gguf/fp16/1/SmolLM-135M-Instruc-fp16.gguf -mg 1 -ngl 10,20,30,35 -n 128,256,512 

| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  20 |          1 |         tg512 |        108.06 ± 3.09 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  30 |          1 |         pp512 |    12402.11 ± 518.29 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  30 |          1 |         tg128 |        164.74 ± 0.57 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  30 |          1 |         tg256 |        163.21 ± 1.35 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  30 |          1 |         tg512 |        157.77 ± 2.31 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  35 |          1 |         pp512 |     13839.79 ± 28.18 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  35 |          1 |         tg128 |        223.03 ± 0.40 |
| llama ?B Q4_K - Medium         | 127.55 MiB |   162.83 M | CUDA       |  3